### `Multinomial Naive Bayes for Text Classification`

A little detour at first.... Consider a piece of text, i.e. a sequence of $n$ tokens $w_1, w_2, ..., w_n$. Assuming a bag of words model, i.e. the ordering of the tokens does not matter, and also assuming that the tokens are conditionally independent and identically distributed, we can express the joint probability of the sequence as follows:

$p(w_1, w_2,...,w_n) = \prod_{i=1}^{n} p(w_i)$

Since there are only a finite number of possibilities for each word, i.e. we have a finite dictionary of words $V$, we can then model each word as being drawn from a `Multinoulli/Categorical distribution`:

$p(w) = \theta_k$ where $\theta_k$ are the parameters of this categorical distribution, and $k\in {1,2,...,|V|}$ is the index corresponding to the word $w$. e.g. if our vocabulary contains 3 words $V: \{cat, dog, apple\}$, then the parameters are  $\{\theta_1, \theta_2, \theta_3\}$ and $p(w=cat)=\theta_1$, similarly $p(w=apple)=\theta_3$. So the parameters are just the probabilities of observing each word from the vocabulary and $\sum_k \theta_k = 1$.

Then we can write:

$p(w_1, w_2,...,w_n) = \prod_{i=1}^{n} p(w_i) = \prod_{i=1}^{n} \theta_{idx(w_i)}$, where $idx(w)$ denotes the index of word $w$.

Beacuse the sequence may contain repeated tokens, it is more convenient in this kind of bag of words model to consider the freqeuncies/counts of each possible word from the vocabulary. We can define $x_k$ as the frequency with which the word at index $k$ of the vocabulary occurs in our sequence. Then for the vocabulary from our previous example, if our sequence is $(cat, cat, dog, cat, dog)$, then we have $x_1 = x_{cat} = 3$, $x_2 = x_{dog}=2$ and $x_3 = x_{apple}=0$. In terms of this frequency description, we can express the joint probability as follows:

$p(w_1, w_2,...,w_n) = \prod_{k=1}^{|V|} (\theta_{k})^{x_k}$

This formula for the joint probability is not totally valid because if we sum the joint probabillities for all possible sequences of some length, it won't sum to 1. This is because our bag of words model treats all possible permutations of any sequence as the same thing, so we need to introduce a multiplicative factor (called the "multinomial coefficient") to compensate for this fact:

$p(w_1, w_2,...,w_n) = \frac{n!}{x_1! x_2!...x_k!} \prod_{k=1}^{|V|} (\theta_{k})^{x_k}$

Now how do we find the parameters? We can estimate them by using `Maximum Likelihood Estimation (MLE)`. Given a training corpus, i.e. a very long sequence, we can choose parameter values that will maximise the joint probability for this sequence, i.e.

$\hat{\theta} = \text{argmax}_{\theta} \text{  } p(sequence) = \text{argmax}_{\theta}  \prod_{k=1}^{|V|} (\theta_{k})^{x_k}$.

Separately for each parameter, we can write down the following:

$\frac{d p}{d\theta_k} = 0$  $\implies \frac{d \log p}{d\theta_k} = 0$ subject to the constraint $\sum_{k=1}^{|V|} \theta_k = 1$.

To solve this constrained optimization problem, we can use `Lagrange multipliers` and extremize the lagrangian $L = \log p + \lambda (1-\sum_{k=1}^{|V|} \theta_k)$:

$\frac{d L}{d\theta_k} = 0, \frac{d L}{d\lambda} = 0$

$\frac{d}{d\theta_k} \sum_{k=1}^{|V|} (x_k \log \theta_k - \lambda \theta_k) + \lambda = 0 \implies \frac{x_k}{\theta_k} - \lambda =0 \implies \theta_k = \frac{x_k}{\lambda}$ 

$\frac{d}{d\lambda} \sum_{k=1}^{|V|} (x_k \log \theta_k - \lambda \theta_k) + \lambda = 0 \implies \sum_{k=1}^{|V|} \theta_k = 1 \implies \sum_{k=1}^{|V|} \frac{x_k}{\lambda} = 1 \implies \lambda = \sum_{k=1}^{|V|} x_k$

So we finally have our maximum likelihood estimate for the parameters: $\hat{\theta}_k = \frac{x_k}{\sum_{k=1}^{|V|} x_k}$. This just says that the probability of observing a word is its relative frequency in the corpus.

Now that we've seen how to estimate parameters of a Multinoiulli distribution, let's look how a `Multinomial Naive Bayes Classifier` works. Given a document, which is a sequence of tokens, i.e. $d = (w_1, w_2,...,w_n)$, we want to predict a label for it $c \in C = \{c_1, c_2, ..., c_m\}$. We can have a probabilitic model that allows us to predict the probabilities for each possible label $c_k$, i.e. $P(c_k|d)$. Then we predict the label by picking the class with the highest probability given the document:

$\hat{c} = \text{argmax}_{c_k \in C} \text{ } P(c_k|d) = \text{argmax}_{c_k \in C} \frac{P(d|c_k)P(c_k)}{P(d)} = \text{argmax}_{c_k \in C} P(d|c_k)P(c_k)$

We used Baye's rule to get the final equation. Note tha Naive Bayes is a generative probabilistic model, i.e. we can think of first sampling a class label $c_k$ from a `prior` distribution $P(c)$, then we generate the sequence d one word at a time from the distribution $P(d|c_k)$, also called the `data likelihood`. Assuming a bag words model and conditional independence between the words allows us to describe $P(d|c_k)$ using a multinomial distribution, we can also use a Multinoulli distribution to model $P(c)$, and learn the parameters of these distributions using MLE given a training corpus which is the set of tuples: $\{(d^{(i)}, c^{(i)})| i=1,2,...N\}$:

$P(c=c_k) = \frac{N_{c_k}}{N}$ where $N$ is the total number of documents in the training corpus and $N_c$ is the number of documents with that class label $c$.

$P(d|c) = P(w_1, w_2, ...,w_n|c) = \prod_{i=1}^{n} p(w_i|c) = \prod_{k=1}^{|V|} \theta_{c,k} = \prod_{k=1}^{|V|} \frac{count(w_k, c)}{\sum_{w \in V}count(w,c)}$

where $count(w,c)$ is the frequency with which word $w$ occurs across all documents with label $c$ in the training corpus. Because $P(d|c)$ contains a product of a large number of terms, in practice, we will work with the log probability to avoid numerical underflow. In practice, we also apply `add-1 smoothing` to the counts to account for out-of-vocabulary words during test time. 

#### `Data Likelihood as a Language Model`

$p(w|c)$ can be viewed as a class-specific unigram language model, i.e. we have a separate language model for each class. If we assume that the training corpus has equal number of documents from each class such that the prior distribution over labels is uniform, i.e. if we have $m$ labels $\{c_1, c_2, ..., c_m\}$, then $P(c)=\frac{1}{m} \forall c$, then the Naive Baye's classification rule is basically equivalent to finding out the probability/likelihood $P(d|c_k)$ of generating a document $d$ under each of these language models and then picking the highest probability class:  $\text{argmax}_{c_k \in C} P(d|c_k)$. A non-uniform prior would slightly slightly modify this by weighting each likelihood, e.g. if a certain label category occurs less frequently, then the likelihood of generating the document under the language model for that class would get a lower weight. 

By giving up the conditional independence assumption between words in a sequence, we can replace the unigram language model with something more powerful, e.g. a bigram language model, and use that to compute the likelihood $P(d|c_k)$ of a document.